[acoli.online](acoli.online) is a site with folktalks, poetry and other resources in the Acholi language. It is a static site so doesn't need to be repeatedly scraped. This notebook yields about 2000 sentences.

In [22]:
import nltk   
import urllib
import re
from bs4 import BeautifulSoup

In [190]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [218]:
sentences = []

ododo_pages = [1170, 1188, 1203, 1023, 640, 637, 626, 850,
         630, 623, 608, 532, 528, 524, 520, 516] # ododo

for page_id in ododo_pages:
    url = "https://acoli.online/?page_id=1203"    
    raw_html = urllib.request.urlopen(url).read()    
    
    html = (str(raw_html).replace('<br>', r'\n'))
    html = html.replace(r'\t', '\n').replace(r'\r', '\n').replace(r'\n', '\n')
    soup = BeautifulSoup(html, 'html.parser')
    text = str(soup.get_text())
    lines = [line for line in text.split('\n') if len(line)>5]
    lines = lines[8:-2]
    sentences.extend(split_into_sentences(lines[0]))  

In [219]:
# poetry
poetry_pages = [346, 365, 392, 395, 397, 399]

for page_id in poetry_pages:
    url = "https://acoli.online/?page_id=346"    
    raw_html = urllib.request.urlopen(url).read()  

    html = (str(raw_html).replace('<br>', '\n'))
    html = html.replace(r'\t', '\n').replace(r'\r', '\n').replace(r'\n', '\n')
    soup = BeautifulSoup(html, 'html.parser')
    text = str(soup.get_text())
    lines = [line for line in text.split('\n') if len(line)>5]
    lines = lines[9:-2]
    sentences.extend(lines)

In [221]:
with open("acholi-online.txt", "w") as f:
    f.writelines('\n'.join(sentences))